In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb

# Load the dataset (replace with your actual file path)
df = pd.read_csv('data/2024-06-01_2024-09-01/aggregated/percent_return.csv')

# Extract stock names
stock_names = df.columns[1:]  # Assuming the first column is the date

print(df) #every 5th row is a friday

          date         A       AAL      AAPL      ABBV      ABNB       ABT  \
0   2024-06-03  0.889681 -0.603454  0.469502 -0.792589  0.349934  0.153790   
1   2024-06-04 -0.481854 -0.346620 -0.264434  0.295955  0.864075  0.081875   
2   2024-06-05  2.123945  0.955688  0.124639  0.634790 -1.486685 -0.237806   
3   2024-06-06  0.086028 -1.295334 -0.733231  0.933931  1.023988  0.416045   
4   2024-06-07  0.593907  1.232397  1.033843 -0.664010  0.314642  3.035119   
..         ...       ...       ...       ...       ...       ...       ...   
58  2024-08-26 -0.439370 -2.109303  0.185217 -0.060739 -0.975609 -0.035481   
59  2024-08-27  0.441351 -0.877194  0.898230 -1.045511  1.248911  0.035380   
60  2024-08-28 -0.332086 -0.196661 -0.627410 -0.061380  1.318964 -0.176832   
61  2024-08-29  0.323778  1.464849 -0.134730 -0.418371 -1.328594 -0.476612   
62  2024-08-30  0.534605  1.239277 -0.516965  0.542892  0.316397  0.238935   

        ACGL       ACN      ADBE  ...       WTW        WY      

In [12]:
# Prepare the data for Monday to Thursday (features) and Friday (target)
X = []
y = []

# Iterate over the dataset in chunks of 5 rows (representing one week)
for i in range(0, len(df), 5):
    if i + 4 >= len(df):  # Prevent going out of bounds
        break
    # Monday to Thursday data (features)
    X.append(df.iloc[i:i+4, 1:].values.flatten())  # Flatten Monday-Thursday into a single row
    # Friday data (target)
    y.append(df.iloc[i+4, 1:].values)  # Friday returns as target

# Convert to numpy arrays
X = np.array(X)
y = np.array(y)

# Train-test split (e.g., 80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
### This block takes a while to run.

# Train an XGBoost regression model
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1)
model.fit(X_train, y_train)

# Evaluate the model on the test set
predictions = model.predict(X_test)

In [16]:
# Get the number of weeks (rows in predictions and y_test)
num_weeks = min(len(predictions), len(y_test))

# Print predictions and actual values
print("\nComparison (Predictions vs Actual):")
for i in range(num_weeks):  # Use the minimum size of predictions and y_test
    print(f"Week {i + 1}:")
    for stock_index in range(len(stock_names)):
        print(f"{stock_names[stock_index]} - Prediction: {float(predictions[i, stock_index])}, Actual: {float(y_test[i, stock_index])}")
    print()  # New line for better readability



Comparison (Predictions vs Actual):
Week 1:
A - Prediction: -5.028044700622559, Actual: -0.6045562351468284
AAL - Prediction: -2.5448033809661865, Actual: -0.1939908530519288
AAPL - Prediction: 2.065013885498047, Actual: 0.327762865027026
ABBV - Prediction: 0.0239272378385067, Actual: -0.4920866704194226
ABNB - Prediction: -0.7695903182029724, Actual: -1.31159000837989
ABT - Prediction: -0.08676589280366898, Actual: -0.9390118535355476
ACGL - Prediction: 0.4992449879646301, Actual: -0.1650948498883254
ACN - Prediction: -0.6721261143684387, Actual: 0.1333623650998701
ADBE - Prediction: -0.9206030964851379, Actual: -0.3950504085013256
ADI - Prediction: -1.4863332509994507, Actual: -0.6007926585032557
ADM - Prediction: 0.6571198105812073, Actual: -1.828859363245072
ADP - Prediction: 0.1457124650478363, Actual: -0.2502286476411474
ADSK - Prediction: -0.546606719493866, Actual: -0.2552136933223686
AEE - Prediction: 0.3501914143562317, Actual: -1.1617239381247126
AEP - Prediction: -0.805242